In [9]:
import sys
sys.path.append("../../src/")
import numpy as np
import math
import uncertainpy.gradual as grad
import random
random.seed(0)

In [10]:
# generate random sublist from current list
def generate_new_list(original_list):
    new_list = []
    for element in original_list:
        if random.choice([True, False]):
            new_list.append(element)
    return new_list

In [11]:
# return cur_arg's SHAP AAE
def compute_SHAP(bag, topic_a, cur_arg):
    model.BAG = grad.BAG("../../bags/td_qbaf.bag")
    model.solve(delta=10e-2, epsilon=10e-4, verbose=False, generate_plot=False)

    arg_name_all = []

    topic_arg = model.BAG.arguments[topic_a]
    for arg in model.BAG.arguments.values():
        arg_name_all.append(arg.name)
        # print((arg.name,arg.strength))
    arg_name_no_topic_arg = arg_name_all.copy()
    arg_name_no_topic_arg.remove(topic_a)

    print(arg_name_all)
    print(arg_name_no_topic_arg)

    # remove current argument
    arg_name_no_topic_no_cur = arg_name_no_topic_arg.copy()
    print(cur_arg)

    arg_name_no_topic_no_cur.remove(cur_arg)
    print(arg_name_no_topic_no_cur)
    print("----------------------------")


    # record initial base scores
    arg_initial_weight = []
    for arg in model.BAG.arguments.values():
        arg_initial_weight.append((arg.name,arg.initial_weight))


    # print all the initial base scores
    print("All the base score and strength in QBAF:")
    for arg in model.BAG.arguments.values():
        print((arg.name,arg.initial_weight,arg.strength))


    sample_count = 1000 # iterate 1000 times
    difference = []
    for i in range(sample_count):

        new_list = generate_new_list(arg_name_no_topic_no_cur) # choose random elements in arg_name_no_topic_arg
        print(f"new_list:{new_list}")

        if new_list == []:
            new_tuple = ()
        else:
            new_tuple = tuple(new_list)
        j = new_tuple

        # remove arg(j) from the current combination i
        for k in j: # first remove the argument from the combination
            model.BAG.remove_argument(k)

        model.solve(delta=10e-2, epsilon=10e-4, verbose=False, generate_plot=False)

        # compute sigma(S U {i})
        # print("Initial and final strength (1st):")
        # for arg in model.BAG.arguments.values():
        #     print((arg.name,arg.initial_weight,arg.strength))
        topic_arg = model.BAG.arguments[topic_a]
        strength_before_drop_cur = topic_arg.strength
        print(f"strength_before_drop_cur:{strength_before_drop_cur}")

        # remove cur arg
        model.BAG.remove_argument(cur_arg)

        model.solve(delta=10e-2, epsilon=10e-4, verbose=False, generate_plot=False)

        # compute sigma(S)
        # print("Initial and final strength (2nd):")
        # for arg in model.BAG.arguments.values():
        #     print((arg.name,arg.initial_weight,arg.strength))
        topic_arg = model.BAG.arguments[topic_a]
        strength_after_drop_cur = topic_arg.strength
        print(f"strength_after_drop_cur:{strength_after_drop_cur}")

        # compute sigma(S U {i})-sigma(S)
        difference.append(strength_before_drop_cur-strength_after_drop_cur)
        print(f"diff={strength_before_drop_cur-strength_after_drop_cur}")


        # recover BAG for the next iteration
        model.BAG = grad.BAG("../../bags/td_qbaf.bag")
        model.solve(delta=10e-2, epsilon=10e-4, verbose=False, generate_plot=False)


    print("----------------------------")

    print(f"difference:{difference}")
    attribution = sum(difference)/sample_count
    print(attribution)

    # reset to original initial weight
    model.BAG = grad.BAG("../../bags/td_qbaf.bag")
    model.solve(delta=10e-2, epsilon=10e-4, verbose=False, generate_plot=False)

    return attribution

In [12]:
model = grad.semantics.QuadraticEnergyModel()
model.approximator = grad.algorithms.RK4(model)
model.BAG = grad.BAG("../../bags/td_qbaf.bag")
model.solve(delta=10e-2, epsilon=10e-4, verbose=True, generate_plot=False)

QuadraticEnergyModel, RK4
Time: 12.199999999999973
Argument(name=s0, weight=0.5, strength=0.664592759656642)
Argument(name=s1, weight=0.5, strength=0.664592759656642)
Argument(name=s2, weight=0.5, strength=0.664592759656642)
Argument(name=s3, weight=0.5, strength=0.8135424356888933)
Argument(name=s4, weight=0.5, strength=0.7041994642766211)
Argument(name=s5, weight=0.5, strength=0.7041994642766211)
Argument(name=s6, weight=0.5, strength=0.5000000446420562)
Argument(name=s7, weight=0.5, strength=0.8254237055377077)
Argument(name=s8, weight=0.5, strength=0.7235959804104234)
Argument(name=s9, weight=0.5, strength=0.7235959804104234)
Argument(name=s10, weight=0.5, strength=0.7235959804104234)
Argument(name=c0, weight=0.0, strength=0.7000991119110755)
Argument(name=c1, weight=0.0, strength=0.46753135307445565)
Argument(name=c2, weight=0.0, strength=0.8313656875032375)
Argument(name=c3, weight=0.0, strength=4.617127656482882e-07)
Argument(name=c4, weight=0.0, strength=3.570538632983414e-07)


0.0009878603392623568

In [13]:
# compute attribution for one arg
topic_a = "c5"
all_args = model.BAG.get_arguments()
all_args_names = []
for i in all_args:
    all_args_names.append(i.get_name())
print(all_args_names)

all_args_names_without_topic = all_args_names.copy()
all_args_names_without_topic.remove(topic_a)
print(all_args_names_without_topic)

attribution = {}
for cur_arg in all_args_names_without_topic:
    attribution[cur_arg] = compute_SHAP(model.BAG, topic_a, cur_arg)
print(f"attribution: \n{attribution}")

['s0', 's1', 's2', 's3', 's4', 's5', 's6', 's7', 's8', 's9', 's10', 'c0', 'c1', 'c2', 'c3', 'c4', 'c5']
['s0', 's1', 's2', 's3', 's4', 's5', 's6', 's7', 's8', 's9', 's10', 'c0', 'c1', 'c2', 'c3', 'c4']
['s0', 's1', 's2', 's3', 's4', 's5', 's6', 's7', 's8', 's9', 's10', 'c0', 'c1', 'c2', 'c3', 'c4', 'c5']
['s0', 's1', 's2', 's3', 's4', 's5', 's6', 's7', 's8', 's9', 's10', 'c0', 'c1', 'c2', 'c3', 'c4']
s0
['s1', 's2', 's3', 's4', 's5', 's6', 's7', 's8', 's9', 's10', 'c0', 'c1', 'c2', 'c3', 'c4']
----------------------------
All the base score and strength in QBAF:
('s0', 0.5, 0.664592759656642)
('s1', 0.5, 0.664592759656642)
('s2', 0.5, 0.664592759656642)
('s3', 0.5, 0.8135424356888933)
('s4', 0.5, 0.7041994642766211)
('s5', 0.5, 0.7041994642766211)
('s6', 0.5, 0.5000000446420562)
('s7', 0.5, 0.8254237055377077)
('s8', 0.5, 0.7235959804104234)
('s9', 0.5, 0.7235959804104234)
('s10', 0.5, 0.7235959804104234)
('c0', 0.0, 0.7000991119110755)
('c1', 0.0, 0.46753135307445565)
('c2', 0.0, 0.83

In [14]:
print(f"base score of the topic_arg: {model.BAG.arguments[topic_a].initial_weight}")
# sum of all attributions
total_sum = sum(attribution.values())
print(f"sum of all attributions: {total_sum}")
print("--------------------------------------------")
print(f"sum of all attributions + BS: {total_sum+model.BAG.arguments[topic_a].initial_weight}")
print(f"strength of the topic_arg: {model.BAG.arguments[topic_a].strength}")

base score of the topic_arg: 0.0
sum of all attributions: 1.0637783193836359
--------------------------------------------
sum of all attributions + BS: 1.0637783193836359
strength of the topic_arg: 0.8996854332891212
